In [1]:
! pip install pymysql

In [4]:
with open("env","r") as file:
    secret = file.readline()


In [3]:
import pymysql.cursors
import pandas as pd
import requests
import json

path_files=""

def Get_DB():
    class Config:
        MYSQL_HOST = "db4free.net"
        MYSQL_PORT = 3306  # default port ของ MySQL คือ 3306
        MYSQL_USER =secret[3:11].strip()
        MYSQL_PASSWORD =secret[17:25].strip()
        MYSQL_DB = #Database Name
        MYSQL_CHARSET = 'utf8mb4'

    connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)
    
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT * FROM online_retail;"
        cursor.execute(sql)
        result = cursor.fetchall() 

    #Convert DB(Type list) to DataFrame(Type pandas.core.frame.DataFrame)
    retail = pd.DataFrame(result) 
    retail.to_csv(path_files+"Raw_Retail_Online.csv", index=False)
        

def Treat_DB():
    retail = pd.read_csv(path_files+"Raw_Retail_Online.csv")
    #Copy Column 'InvoiceTimestamp' from 'InvoiceDate' ในรูปแบบ datetime
    retail['InvoiceTimestamp'] = pd.to_datetime(retail['InvoiceDate'])
    #Convert Type 'InvoiceDate' to datetime
    retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate']).dt.date
    retail.to_csv(path_files+"TR_Retail_Online.csv", index=False)

def Connect_API():
    url = "https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all"
    #Get API
    response = requests.get(url)
    #Convert API(Type str) to json(Type Json)
    result_conversion_rate = response.json()
    #Convert json(Type Json) to DataFrame(Type pandas.core.frame.DataFrame)
    conversion_rate = pd.DataFrame.from_dict(result_conversion_rate)
        #Rename Column "index" to "date" ตอนAPI ไม่มีหัวColumnมาให้ ต้องแปลงเพื่อใช้รวมตาราง
    conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})
    conversion_rate.to_csv(path_files+"Raw_API.csv", index=False)

def Treat_API(): 
    conversion_rate = pd.read_csv(path_files+"Raw_API.csv")
    #Convert type of "date" from "datetime" to type "date"
    conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date
    conversion_rate.to_csv(path_files+"TR_API.csv", index=False)

def Merge():
    retail = pd.read_csv(path_files+"TR_Retail_Online.csv")
    conversion_rate = pd.read_csv(path_files+"TR_API.csv")
    #Join DataFrame "retail" กับ "conversion_rate" แบบ "left" โดยดูจาก "InvoiceDate"ให้ตรงกับ "date"
    final_df = retail.merge(conversion_rate, how="left", left_on="InvoiceDate", right_on="date")
    final_df.to_csv(path_files+"MR_Retail_Online.csv", index=False)

def Convert_rate():
    final_df = pd.read_csv(path_files+"MR_Retail_Online.csv")
    #สร้าง Column "THBPrice" ขึ้นจาก "UnitPrice" x "Rate"
    final_df["THBPrice"]=final_df["UnitPrice"] * final_df["Rate"]
    final_df.to_csv(path_files+"Final_Retail_Online.csv", index=False)

Get_DB()
Treat_DB()
Connect_API()
Treat_API()
Merge()
Convert_rate()